<a href="https://colab.research.google.com/github/lakshmisharma17/module20/blob/main/Capstone_Module20_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

# Load the dataset
df = pd.read_csv('sample_data/housing.csv')

# Step 1: Handle Missing Values

# Identify missing values
print("Missing values before handling:")
display(df.isnull().sum())

# Drop rows with any missing values
df_cleaned = df.dropna()

print("\nMissing values after handling:")
display(df_cleaned.isnull().sum())

# Display the shape of the data before and after dropping missing values
print("\nShape before dropping missing values:", df.shape)
print("Shape after dropping missing values:", df_cleaned.shape)

Missing values before handling:


,0
longitude,0
latitude,0
housing_median_age,0
total_rooms,0
total_bedrooms,207
population,0
households,0
median_income,0
median_house_value,0
ocean_proximity,0



Missing values after handling:


,0
longitude,0
latitude,0
housing_median_age,0
total_rooms,0
total_bedrooms,0
population,0
households,0
median_income,0
median_house_value,0
ocean_proximity,0



Shape before dropping missing values: (20640, 10)
Shape after dropping missing values: (20433, 10)
